In [4]:
import os
import numpy as np # to process the data
import random
import math
from PIL import Image
from sklearn.model_selection import train_test_split

In [5]:
class FlowerDataset:
    def __init__(self, directory):
        self.directory = directory
        self.images = []
        self.labels = []

    def load_data(self):
        for i, img in enumerate(os.listdir(self.directory)):
            if i >= 10:  # only load the first 10 images
                break
            try:
                img_path = os.path.join(self.directory, img)
                with Image.open(img_path) as img_array:  # read the image
                    gray_array = img_array.convert('L')  # ensure image is grayscale
                    resized_array = gray_array.resize((150, 150))  # resize the image
                    self.images.append(list(resized_array.getdata()))  # keep image in 1D
                    label = img.split('_')[0]  # extract label from filename
                    self.labels.append(label)
            except Exception as e:
                print(f"Error loading image {img}: {e}")

    def preprocess_data(self):
        # Normalize pixel values
        self.images = [[pixel / 255.0 for pixel in image] for image in self.images]
        print(f"Loaded {len(self.images)} images")

# Load and preprocess the data
dataset = FlowerDataset('flowers')
dataset.load_data()
dataset.preprocess_data()

# Print the shape of the preprocessed images
print(f"Shape of the preprocessed images: {len(dataset.images)} images, each with {len(dataset.images[0])} pixels")

# Dimensions of the first preprocessed image
print(f"Dimensions of the first preprocessed image: {len(dataset.images[0])} pixels")

# Split the data into training, validation, and test sets
train_ratio = 0.75
validation_ratio = 0.15
test_ratio = 0.10

# Custom function to split the dataset
def custom_train_test_split(data, labels, train_size):
    train_count = int(len(data) * train_size)
    return data[:train_count], data[train_count:], labels[:train_count], labels[train_count:]

# train is now 75% of the entire data set
x_train, x_temp, y_train, y_temp = custom_train_test_split(dataset.images, dataset.labels, train_ratio)

# test is now 10% of the initial data set, validation is now 15%
val_count = int(len(x_temp) * (validation_ratio / (test_ratio + validation_ratio)))
x_val, x_test = x_temp[:val_count], x_temp[val_count:]
y_val, y_test = y_temp[:val_count], y_temp[val_count:]

# Print the sizes of the datasets
print(f"Train set size: {len(x_train)}, Validation set size: {len(x_val)}, Test set size: {len(x_test)}")

# Print the unique classes in the training set
unique_classes = set(y_train)
print(f"Unique classes in the training set: {unique_classes}")

Loaded 10 images
Shape of the preprocessed images: 10 images, each with 22500 pixels
Dimensions of the first preprocessed image: 22500 pixels
Train set size: 7, Validation set size: 1, Test set size: 2
Unique classes in the training set: {'peonies', 'hydrangeas', 'daisies', 'hibiscus', 'tulip', 'orchids', 'lilies'}


In [25]:
# Base Layer class
class Layer:
    def forward_propagation(self, input_data):
        raise NotImplementedError

    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

# Convolutional Layer
class ConvLayer(Layer):
    def __init__(self, num_filters, filter_size):
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.filters = [[0.01 * random.random() for _ in range(filter_size)] for _ in range(num_filters)]

    def forward_propagation(self, input_data):
        self.input_data = input_data
        size = len(input_data)  # Get the size of the 1D image
        output_data = [0.0 for _ in range(size - self.filter_size + 1)]
        for i in range(size - self.filter_size + 1):
            for f in range(self.num_filters):
                output_data[i] += sum(input_data[i + di] * self.filters[f][di] for di in range(self.filter_size))
        return output_data

    def backward_propagation(self, output_error, learning_rate):
        d_filters = [[0.0 for _ in range(self.filter_size)] for _ in range(self.num_filters)]
        size = len(self.input_data)
        for i in range(size - self.filter_size + 1):
            for f in range(self.num_filters):
                for di in range(self.filter_size):
                    d_filters[f][di] += output_error[i] * self.input_data[i + di]
        for f in range(self.num_filters):
            for di in range(self.filter_size):
                self.filters[f][di] -= learning_rate * d_filters[f][di]
        return None  # No error propagation to previous layer
        
# MaxPooling Layer
class MaxPoolingLayer(Layer):
    def __init__(self, pool_size):
        self.pool_size = pool_size

    def forward_propagation(self, input_data):
        # ... (max pooling implementation)
        pass

    def backward_propagation(self, output_error, learning_rate):
        # ... (backpropagation for max pooling)
        pass
class FlattenLayer(Layer):
    def forward_propagation(self, input_data):
        self.input_shape = len(input_data)  # Save the input shape (it will be needed in backpropagation)
        return [item for sublist in input_data for item in sublist]  # Flatten the input data

    def backward_propagation(self, output_error, learning_rate):
        # Reshape the output error to the original input shape
        return [output_error[i * self.input_shape + j] for j in range(self.input_shape) for i in range(self.input_shape)]
        
# Dropout Layer
class DropoutLayer(Layer):
    def __init__(self, dropout_rate):
        self.dropout_rate = dropout_rate

    def forward_propagation(self, input_data):
        # ... (dropout implementation)
        pass

    def backward_propagation(self, output_error, learning_rate):
        # ... (backpropagation for dropout)
        pass

# Dense Layer
class DenseLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = [[0.01 * random.random() for _ in range(input_size)] for _ in range(output_size)]

    def forward_propagation(self, input_data):
        # ... (dense layer implementation)
        pass

    def backward_propagation(self, output_error, learning_rate):
        # ... (backpropagation for dense layer)
        pass

class SoftmaxLayer(Layer):
    def forward_propagation(self, input_data):
        exps = [math.exp(i) for i in input_data]
        return [exp_i / sum(exps) for exp_i in exps]  # Softmax function

    def backward_propagation(self, output_error, learning_rate):
        return output_error  # The gradient of the softmax function is the output error itself

# ReLU Activation
class ReLULayer(Layer):
    def forward_propagation(self, input_data):
        # ... (ReLU implementation)
        pass

    def backward_propagation(self, output_error, learning_rate):
        # ... (backpropagation for ReLU)
        pass

# Cross-Entropy Loss
class CrossEntropyLoss:
    def calculate_loss(self, predicted, actual):
        # ... (cross-entropy loss implementation)
        pass

    def calculate_gradient(self, predicted, actual):
        # ... (gradient calculation for cross-entropy loss)
        pass


In [26]:
# Define the CNN model
class CNN:
    def __init__(self):
        self.layers = [
            ConvLayer(num_filters=8, filter_size=3),  # Convolutional layer
            ReLULayer(),  # Activation layer
            MaxPoolingLayer(pool_size=2),  # Pooling layer
            FlattenLayer(),  # Flatten layer
            DenseLayer(input_size=8*74*74, output_size=len(unique_classes)),  # Dense layer
            SoftmaxLayer()  # Softmax layer
        ]

    def forward_propagation(self, input_data):
        for layer in self.layers:
            input_data = layer.forward_propagation(input_data)
        return input_data

    def backward_propagation(self, output_error, learning_rate):
        for layer in reversed(self.layers):
            output_error = layer.backward_propagation(output_error, learning_rate)

    def train(self, x_train, y_train, epochs, learning_rate):
        for epoch in range(epochs):
            error = 0
            for x, y in zip(x_train, y_train):
                output = self.forward_propagation(x)
                error += cross_entropy_loss.calculate_loss(output, y)
                output_error = cross_entropy_loss.calculate_gradient(output, y)
                self.backward_propagation(output_error, learning_rate)
            error /= len(x_train)
            print(f'Epoch {epoch+1}/{epochs}, Loss: {error}')

    def predict(self, input_data):
        output = self.forward_propagation(input_data)
        return output.index(max(output))  # Return the index of the max value

# Create the CNN model
cnn = CNN()

# Train the model
cnn.train(x_train, y_train, epochs=10, learning_rate=0.01)


TypeError: object of type 'NoneType' has no len()